Вычислить с заданой погрешностью решение задачи Коши

$\left\{ \begin{gathered} \frac{dy}{dx} = -\frac{y^2 + 4x(x + 1)}{y} \\ y(1) = 12 \end{gathered}\right. ,\space \epsilon = 10^{-4}, \space x \in (1, 2)$

Аналитическое решение данного уравнения:

$y(x) = 2 \cdot \sqrt{x^2 + 35e^{2 - 2x}} $



In [17]:
import numpy as np
import math

In [18]:
def solve(x):
    return 2*np.sqrt(x**2 + 35*np.e**(2 - 2*x))

solve(2)

5.911593664412819

In [19]:
a = 1
b = 2
eps = 1e-4
k = 1
y0 = 12
target = np.linspace(a, b, 11)

def f(x,y):
    if y != 0:
        return -(y**2 + 4*x*(x + 1))/y
    else:
        print('error')
        return 0

In [20]:
def genX(n):
    xx = []
    for i in range(10):
        xx.extend(list(np.linspace(target[i], target[i+1], n, endpoint=False)))

    xx.append(b)
    
    return xx
    
def getStep(xx):
    h = (b-a)/(len(xx)-1)
    return h

In [21]:
def iterate(x, y, h):
    f1 = f(x,y)
    f2 = f(x + h/3, y + h/3*f1)
    f3 = f(x + 2*h/3, y + 2*h/3*f2)
    return y + h/4*(f1 + 3*f3)

In [22]:
def rk(xx, h):
    x = a
    y_prev = y0
    y_curr = y0

    res = []
    for x in xx:
        for t in target:
            if abs(t-x)<1e-8:
                res.append(y_curr)
        y_prev = y_curr
        y_curr = iterate(x, y_curr, h)

        yh = iterate(x, y_curr, h)
        y2h = iterate(x, y_curr, 2*h)
        # assert abs(y2h - yh) <= (2**k - 1) * eps
    return res

In [23]:
xx = genX(4096)
res = rk(xx, getStep(xx))

In [24]:
xxmax = genX(2**20)
resmax = rk(xxmax, getStep(xxmax))

In [25]:
len(xxmax)

10485761

In [26]:
diff = []
for i in range(len(res)):
    diff.append((res[i]-resmax[i]))

In [27]:
import pandas as pd
pd.DataFrame({'x': target, 'y': res, 'y*': resmax, 'd': diff}).transpose()

,0,1,2,3,4,5,6,7,8,9,10
x,1.0,1.100000e+00,1.200000e+00,1.300000e+00,1.400000e+00,1.500000e+00,1.600000e+00,1.700000e+00,1.800000e+00,1.900000e+00,2.000000e+00
y,12.0,1.078574e+01,9.666818e+00,8.629260e+00,7.659026e+00,6.741377e+00,5.859756e+00,4.993631e+00,4.113476e+00,3.166107e+00,2.007392e+00
y*,12.0,1.078574e+01,9.666818e+00,8.629260e+00,7.659026e+00,6.741377e+00,5.859756e+00,4.993631e+00,4.113476e+00,3.166107e+00,2.007392e+00
d,0.0,9.610091e-13,8.135714e-13,1.094236e-12,1.090683e-12,8.526513e-13,1.066702e-12,1.285194e-12,8.517631e-13,9.965362e-13,1.644462e-12


In [28]:
N = []
d = []

for i in range(18):
    xx = genX(2**i)
    res = rk(xx, getStep(xx))
    norm = 0
    for j in range(len(res)):
        norm = max(abs(res[j]-resmax[j]), norm)
    d.append(norm)
    N.append(2**i)

In [29]:
rate = [0]
for i in range(len(d)-1):
    rate.append(d[i]/d[i+1])

rate

[0,
 5.842638654607297,
 6.933987024459049,
 7.530509750730704,
 7.794211590724498,
 7.906066890594131,
 7.955268659055196,
 7.976583338680392,
 7.975222228903287,
 7.88288386147511,
 7.197626070578201,
 4.499360409338023,
 1.6889008911693222,
 1.056189389617798,
 0.9980074010816966,
 1.1047169811320754,
 1.3327745180217938,
 0.5042265426880812]

In [30]:
    pd.DataFrame({'N': N, 'd': d, 'rate': rate})

,N,d,rate
0,1,6.745543e-03,0.000000
1,2,1.154537e-03,5.842639
2,4,1.665041e-04,6.933987
3,8,2.211060e-05,7.530510
4,16,2.836797e-06,7.794212
5,32,3.588127e-07,7.906067
6,64,4.510379e-08,7.955269
7,128,5.654524e-09,7.976583
8,256,7.090115e-10,7.975222
9,512,8.994316e-11,7.882884


In [31]:
norm = max(diff)
norm

1.6444623440747819e-12